In [101]:
# In Jupyter cell 1:
import hashlib

# Constants and initializations
CHARSET = "qpzry9x8gf2tvdw0s3jn54khce6mua7l"

pubkey_hex = "0279be667ef9dcbbac55a06295ce870b07029bfcdb2dce28d959f2815b16f81798"
hrp = "bc"
witver = 0 # 0 for P2WPKH, 1 for P2WSH  (not used in this example)
#BECH32M_CONST = 0x2bc830a3

In [102]:
# In Jupyter cell 2:
# Compute P2WPKH witness program from public key
pubkey_bytes = bytes.fromhex(pubkey_hex)
sha256_hash = hashlib.sha256(pubkey_bytes).digest()
ripemd160_hash = hashlib.new("ripemd160", sha256_hash).digest()
witprog = ripemd160_hash
witprog


b'u\x1ev\xe8\x19\x91\x96\xd4T\x94\x1cE\xd1\xb3\xa3#\xf1C;\xd6'

In [103]:
# In Jupyter cell 5:
# Convert to base32
# the padding is not needed because input of ripemd160 is 160 bits, which is divible by 5
acc = 0
bits = 0
ret = []
maxv = (1 << 5) - 1 # 5 bits minus 1 max 32
max_acc = (1 << (8 + 5 - 1)) - 1 # 12 bits, 4 bits left and the new 8 bits
for value in witprog:
    acc = ((acc << 8) | value) & max_acc
    bits += 8
    while bits >= 5:
        bits -= 5
        ret.append((acc >> bits) & maxv)
converted = ret
print(converted)

[14, 20, 15, 7, 13, 26, 0, 25, 18, 6, 11, 13, 8, 21, 4, 20, 3, 17, 2, 29, 3, 12, 29, 3, 4, 15, 24, 20, 6, 14, 30, 22]


In [104]:
# In Jupyter cell 3:
# Utility for checksum
hrp_expanded = [ord(x) >> 5 for x in hrp] + [0] + [ord(x) & 31 for x in hrp]
hrp_expanded = hrp_expanded + [witver] + converted
print(hrp_expanded)




[3, 3, 0, 2, 3, 0, 14, 20, 15, 7, 13, 26, 0, 25, 18, 6, 11, 13, 8, 21, 4, 20, 3, 17, 2, 29, 3, 12, 29, 3, 4, 15, 24, 20, 6, 14, 30, 22]


In [105]:
# In Jupyter cell 4:
# Compute checksum
generator = [0x3b6a57b2, 0x26508e6d, 0x1ea119fa, 0x3d4233dd, 0x2a1462b3]
chk = 1
values_for_checksum = hrp_expanded + [0] * 6  # appending zeros for checksum space
for value in values_for_checksum:
    top = chk >> 25
    chk = (chk & 0x1ffffff) << 5 ^ value
    for i in range(5):
        chk ^= generator[i] if ((top >> i) & 1) else 0
polymod = chk ^ 1

checksum = [(polymod >> 5 * (5 - i)) & 31 for i in range(6)]
checksum


[12, 7, 9, 17, 11, 21]

In [106]:
# In Jupyter cell 6:
# Construct the final encoded string
encoded_data = [0] + converted + checksum
segwit_address = hrp + '1' + ''.join([CHARSET[d] for d in encoded_data])
segwit_address


'bc1qw508d6qejxtdg4y5r3zarvary0c5xw7kv8f3t4'